## Example

In [1]:
from pyrb import EqualRiskContribution, RiskBudgeting, RiskBudgetAllocation
import pandas as pd
import numpy as np

In [2]:
#get a covariance matrix of an asset universe
covariance_matrix = pd.read_csv("data.csv",sep=";",index_col=0).pct_change().cov() * 260
covariance_matrix

,US BONDS 10Y,GERMAN BONDS 10Y,S&P 500,EUROSTOXX 50,NIKKEI,MSCI Emerging,Commodities (CRB),Iboxx HY US,Iboxx HY EUR,Emerging Debt
US BONDS 10Y,0.004116,0.002149,-0.003859,-0.005064,-0.004085,-0.003381,-0.001574,-0.000879,-0.000245,0.000484
GERMAN BONDS 10Y,0.002149,0.003150,-0.002809,-0.005692,-0.003468,-0.003604,-0.001310,-0.000996,-0.000262,0.000280
S&P 500,-0.003859,-0.002809,0.042571,0.030959,0.029899,0.024753,0.008173,0.009952,0.000856,0.004332
EUROSTOXX 50,-0.005064,-0.005692,0.030959,0.064347,0.027036,0.031997,0.011019,0.009402,0.003169,0.006211
NIKKEI,-0.004085,-0.003468,0.029899,0.027036,0.060668,0.031786,0.009928,0.010870,0.001777,0.003836
MSCI Emerging,-0.003381,-0.003604,0.024753,0.031997,0.031786,0.058415,0.014405,0.011145,0.003468,0.007879
Commodities (CRB),-0.001574,-0.001310,0.008173,0.011019,0.009928,0.014405,0.031496,0.005023,0.001489,0.002312
Iboxx HY US,-0.000879,-0.000996,0.009952,0.009402,0.010870,0.011145,0.005023,0.011670,0.001523,0.002549
Iboxx HY EUR,-0.000245,-0.000262,0.000856,0.003169,0.001777,0.003468,0.001489,0.001523,0.004500,0.001282
Emerging Debt,0.000484,0.000280,0.004332,0.006211,0.003836,0.007879,0.002312,0.002549,0.001282,0.008640


#### Solving the ERC problem

In [3]:
ERC = EqualRiskContribution(covariance_matrix)
ERC.solve()

 The optimal solution that gives equal risk contributions is:

In [4]:
optimal_weights =  ERC.x
risk_contributions =  ERC.get_risk_contributions(scale = False)
risk_contributions_scaled =  ERC.get_risk_contributions()
allocation = pd.DataFrame(np.concatenate([[optimal_weights,risk_contributions,risk_contributions_scaled]]  ).T, index = covariance_matrix.index,columns=["optinal weigths","risk contribution","risk contribution(scaled)"])
allocation

,optinal weigths,risk contribution,risk contribution(scaled)
US BONDS 10Y,0.224478,0.004225,0.1
GERMAN BONDS 10Y,0.275145,0.004225,0.1
S&P 500,0.039225,0.004225,0.1
EUROSTOXX 50,0.035893,0.004225,0.1
NIKKEI,0.034282,0.004225,0.1
MSCI Emerging,0.029916,0.004225,0.1
Commodities (CRB),0.054598,0.004225,0.1
Iboxx HY US,0.071110,0.004225,0.1
Iboxx HY EUR,0.152814,0.004225,0.1
Emerging Debt,0.082539,0.004225,0.1


Each asset has a risk contribution of 10% to the total risk. We also verify that the sum of the risk budget is equal to the variance:

In [5]:
np.round(np.dot(np.dot(ERC.x,covariance_matrix),ERC.x)**0.5,10) == np.round(allocation['risk contribution'].sum(),10)

True

#### Solving the risk budgeting problem


Now we want the risk contributions equal to specific budgets

In [6]:
budgets = [0.1,0.1,0.1,0.2,0.2,0.05,0.05,0.05,0.05,0.1]
RB = RiskBudgeting(covariance_matrix,budgets)
RB.solve()

In [7]:
optimal_weights =  RB.x
risk_contributions =  RB.get_risk_contributions(scale = False)
risk_contributions_scaled =  RB.get_risk_contributions()
allocation = pd.DataFrame(np.concatenate([[optimal_weights,risk_contributions,risk_contributions_scaled]]  ).T, index = covariance_matrix.index,columns=["optinal weigths","risk contribution","risk contribution(scaled)"])
allocation

,optinal weigths,risk contribution,risk contribution(scaled)
US BONDS 10Y,0.245021,0.004453,0.10
GERMAN BONDS 10Y,0.304664,0.004453,0.10
S&P 500,0.037884,0.004453,0.10
EUROSTOXX 50,0.064091,0.008906,0.20
NIKKEI,0.061810,0.008906,0.20
MSCI Emerging,0.016443,0.002226,0.05
Commodities (CRB),0.035707,0.002226,0.05
Iboxx HY US,0.041859,0.002226,0.05
Iboxx HY EUR,0.103390,0.002226,0.05
Emerging Debt,0.089131,0.004453,0.10


Again the risk contributions match the budgets and the variance equals the sum of the risk contribution.

In [8]:
 np.round(np.dot(np.dot(RB.x,covariance_matrix),RB.x)**0.5,10) == np.round(allocation['risk contribution'].sum(),10)

True